<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# 安裝相關套件

In [1]:
# 安裝 行情 API sdk
!pip install fugle-realtime

     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 54 kB 2.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
# 取得 fugle http API 
from fugle_realtime import HttpClient

import datetime
import pandas as pd
import requests
import time
import threading

# define notify function

In [45]:
class notify_setting():
    
    def __init__(self, api_token, line_token):
        
        self.api_token = api_token
        self.line_token = line_token

    def lineNotifyMessage(self, msg):
    
        headers = {
           "Authorization": "Bearer " + self.line_token, 
           "Content-Type" : "application/x-www-form-urlencoded"
       }

        payload = {'message': msg}
        r = requests.post("https://notify-api.line.me/api/notify", headers = headers, params = payload)
        return r.status_code
        
    def price_info(self, symbol_id):
        
        api_client = HttpClient(api_token=self.api_token)
        quote_data = api_client.intraday.quote(symbolId = symbol_id)['data']['quote']
        
        open_price = quote_data['priceOpen']['price']
        trade_price = quote_data['trade']['price']
        change_rate = quote_data['changePercent']
        
        url = f"https://www.fugle.tw/ai/{symbol_id}"
        
        self.open_price = open_price
        self.trade_price = trade_price
        self.change_rate = change_rate
        self.url = url
        
        
    def price_change_strategy(self, symbol_id, up_rate, down_rate):
        
        while True:
            
            self.price_info(symbol_id)
            
            if self.change_rate >= up_rate:
                
                self.lineNotifyMessage(f"\n\n OH！\n {symbol_id} 現在價格 {self.trade_price} 元 \n 漲跌幅 {self.change_rate} % \n 漲幅已超過 {str(up_rate)} % \n {self.url}")
                print('已送出提醒！')
                break                
                
            elif self.change_rate <= down_rate:
                
                self.lineNotifyMessage(f"\n\n OH！\n {symbol_id} 現在價格 {self.trade_price} 元 \n 漲跌幅 {self.change_rate} % \n 跌幅已超過 {str(-down_rate)} % \n {self.url}")
                print('已送出提醒！')
                break                
                
            else:
                print('Nothing')
                time.sleep(3)
    
    def price_strategy(self, symbol_id ,up_price, down_price):
        
        while True:
            
            self.price_info(symbol_id)
            
            if self.trade_price >= up_price:
                
                self.lineNotifyMessage(f"\n\n OH！\n {symbol_id} 現在價格 {self.trade_price} 元 \n 漲跌幅 {self.change_rate} % \n 已超過目標價 {str(up_price)} 元 \n {self.url}")
                print('已送出提醒！')
                break
                
            elif self.trade_price <= down_price:
               
                self.lineNotifyMessage(f"\n\n OH！\n {symbol_id} 現在價格 {self.trade_price} 元 \n 漲跌幅 {self.change_rate} % \n 已低於目標價 {str(down_price)} 元 \n {self.url}")
                print('已送出提醒！')
                break
                
            else:
                print('nothing')
                time.sleep(3)

In [46]:
# Set Your Fugle API Token from https://developer.fugle.tw/
api_token = 'YOUR_API_TOKEN'

# Set Your line Token from https://notify-bot.line.me/my/
line_token = 'YOUR_LINE_TOKEN'


# 參數設定

In [52]:
symbol_id = '2314'

# 目標價上界訂定
up_price = 75

# 目標價下界訂定
down_price = 70

# 漲跌幅上界訂定
up_changerate = 8

# 漲跌幅下界訂定
down_changerate = -1

# 執行策略

In [53]:
line = notify_setting(api_token=api_token,
                      line_token=line_token)

In [51]:
# 執行策略一：目標價提醒

strategy_1 = threading.Thread(target=line.price_strategy, args=[symbol_id, up_price, down_price])
strategy_1.start()

In [50]:
# 執行策略二：漲跌幅提醒

strategy_2 = threading.Thread(target=line.price_change_strategy, args=[symbol_id, up_changerate, down_changerate])
strategy_2.start()